这章主要讲的是slam 的3D重建，就是通过 深度图+单目 或者其他形式的组合获得对应3D图

octomap_mapping.cpp 这个脚本主要是用来创建一个 3D 环境的八叉树地图（Octomap）。



In [2]:
import numpy as np
import open3d as o3d
import cv2
import os
from scipy.spatial.transform import Rotation as R

def read_pose(file_path):
    poses = []
    with open(file_path, 'r') as f:
        for line in f:
            data = list(map(float, line.strip().split()))
            T = np.eye(4)
            T[:3, 3] = data[:3]
            T[:3, :3] = R.from_quat(data[3:]).as_matrix()
            poses.append(T)
    return poses

def create_point_cloud(color_img, depth_img, K, depth_scale):
    cx, cy = K[0, 2], K[1, 2]
    fx, fy = K[0, 0], K[1, 1]
    
    points = []
    colors = []
    
    for v in range(depth_img.shape[0]):
        for u in range(depth_img.shape[1]):
            d = depth_img[v, u]
            if d == 0 or d >= 7000:
                continue
            
            z = d / depth_scale
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            
            points.append([x, y, z])
            colors.append(color_img[v, u] / 255.0)
    
    return np.array(points), np.array(colors)

def main():
    # Camera parameters
    cx, cy = 325.5, 253.5
    fx, fy = 518.0, 519.0
    depth_scale = 1000.0
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    # Read poses
    poses = read_pose('./dense_RGBD/data/pose.txt')

    # Create octree
    octree = o3d.geometry.Octree(max_depth=16)

    for i in range(5):
        print(f"Processing image {i+1}")
        color_img = cv2.imread(f'./dense_RGBD/data/color/{i+1}.png')
        depth_img = cv2.imread(f'./dense_RGBD/data/depth/{i+1}.pgm', -1)
        
        points, colors = create_point_cloud(color_img, depth_img, K, depth_scale)
        
        # Transform points to world coordinate
        points = (poses[i][:3, :3] @ points.T + poses[i][:3, 3:]).T
        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        
        # Insert point cloud to octree
        octree.convert_from_point_cloud(pcd, size_expand=0.01)

    # Save octree
    o3d.io.write_octree("octomap.octree", octree)
    print("Octree saved as octomap.octree")

if __name__ == "__main__":
    main()


Processing image 1


: 

In [1]:
import numpy as np
import open3d as o3d
import cv2
import os
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm

def read_pose(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            data = list(map(float, line.strip().split()))
            T = np.eye(4)
            T[:3, 3] = data[:3]
            T[:3, :3] = R.from_quat(data[3:]).as_matrix()
            yield T

def create_point_cloud(color_img, depth_img, K, depth_scale):
    cx, cy = K[0, 2], K[1, 2]
    fx, fy = K[0, 0], K[1, 1]
    
    points = []
    colors = []
    
    for v in range(0, depth_img.shape[0], 2):  # 降采样
        for u in range(0, depth_img.shape[1], 2):
            d = depth_img[v, u]
            if d == 0 or d >= 7000:
                continue
            
            z = d / depth_scale
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            
            points.append([x, y, z])
            colors.append(color_img[v, u] / 255.0)
    
    return np.array(points), np.array(colors)

def process_image(i, pose, K, depth_scale):
    print(f"Processing image {i+1}")
    color_img = cv2.imread(f'./dense_RGBD/data/color/{i+1}.png')
    depth_img = cv2.imread(f'./dense_RGBD/data/depth/{i+1}.pgm', -1)
    
    points, colors = create_point_cloud(color_img, depth_img, K, depth_scale)
    
    # Transform points to world coordinate
    points = (pose[:3, :3] @ points.T + pose[:3, 3:]).T
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    return pcd

def main():
    # Camera parameters
    cx, cy = 325.5, 253.5
    fx, fy = 518.0, 519.0
    depth_scale = 1000.0
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    # Read poses
    poses = list(read_pose('./dense_RGBD/data/pose.txt'))

    # Create octree
    octree = o3d.geometry.Octree(max_depth=16)

    # Process images and update octree
    for i, pose in tqdm(enumerate(poses[:5]), total=5):
        pcd = process_image(i, pose, K, depth_scale)
        octree.convert_from_point_cloud(pcd, size_expand=0.01)

    # Save octree
    o3d.io.write_octree("octomap.octree", octree)
    print("Octree saved as octomap.octree")

if __name__ == "__main__":
    main()


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
  0%|          | 0/5 [00:00<?, ?it/s]

: 

In [ ]:
# 垃圾 mac ，一点都跑不了额

In [1]:
import numpy as np
import cv2
import open3d as o3d
from scipy.spatial.transform import Rotation as R
import os

def read_pose(file_path):
    poses = []
    with open(file_path, 'r') as f:
        for line in f:
            data = list(map(float, line.strip().split()))
            T = np.eye(4)
            T[:3, 3] = data[:3]
            T[:3, :3] = R.from_quat(data[3:]).as_matrix()
            poses.append(T)
    return poses

def create_point_cloud(color_img, depth_img, K, depth_scale, T):
    cx, cy = K[0, 2], K[1, 2]
    fx, fy = K[0, 0], K[1, 1]
    
    points = []
    colors = []
    
    for v in range(depth_img.shape[0]):
        for u in range(depth_img.shape[1]):
            d = depth_img[v, u]
            if d == 0 or d >= 7000:
                continue
            
            z = d / depth_scale
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            
            point = np.array([x, y, z, 1])
            point_world = T @ point
            
            points.append(point_world[:3])
            colors.append(color_img[v, u][::-1] / 255.0)  # BGR to RGB
    
    return np.array(points), np.array(colors)

def main():
    # Camera parameters
    cx, cy = 325.5, 253.5
    fx, fy = 518.0, 519.0
    depth_scale = 1000.0
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    # Read poses
    poses = read_pose('./dense_RGBD/data/pose.txt')

    # Create point cloud
    pcd = o3d.geometry.PointCloud()

    for i in range(5):
        print(f"Processing image {i+1}")
        color_img = cv2.imread(f'./dense_RGBD/data/color/{i+1}.png')
        depth_img = cv2.imread(f'./dense_RGBD/data/depth/{i+1}.pgm', -1)
        
        points, colors = create_point_cloud(color_img, depth_img, K, depth_scale, poses[i])
        
        current_pcd = o3d.geometry.PointCloud()
        current_pcd.points = o3d.utility.Vector3dVector(points)
        current_pcd.colors = o3d.utility.Vector3dVector(colors)
        
        # Statistical outlier removal
        current_pcd, _ = current_pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)
        
        pcd += current_pcd

    print(f"Point cloud has {len(pcd.points)} points before downsampling")

    # Voxel downsampling
    pcd = pcd.voxel_down_sample(voxel_size=0.01)

    print(f"Point cloud has {len(pcd.points)} points after downsampling")

    # Save point cloud
    o3d.io.write_point_cloud("map.pcd", pcd)
    print("Point cloud saved as map.pcd")

if __name__ == "__main__":
    main()


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Processing image 1
Processing image 2
Processing image 3
Processing image 4
Processing image 5
Point cloud has 895249 points before downsampling
Point cloud has 448493 points after downsampling
Point cloud saved as map.pcd



下面这个是深度+单目的

In [ ]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as R

# 参数
BOARDER = 20
WIDTH = 640
HEIGHT = 480
FX = 481.2
FY = -480.0
CX = 319.5
CY = 239.5
NCC_WINDOW_SIZE = 2
NCC_AREA = (2 * NCC_WINDOW_SIZE + 1) * (2 * NCC_WINDOW_SIZE + 1)
MIN_COV = 0.1
MAX_COV = 10


def read_dataset_files(path):
    color_image_files = []
    poses = []
    try:
        with open(f"{path}/first_200_frames_traj_over_table_input_sequence.txt", 'r') as fin:
            for line in fin:
                data = line.strip().split()  #这快有问题呀
                image = data[0]
                pose_data = [float(x) for x in data[1:]]
                color_image_files.append(f"{path}/color/{image}")
                r = R.from_quat([pose_data[3], pose_data[4], pose_data[5], pose_data[6]])
                t = np.array(pose_data[:3])
                T = np.eye(4)
                T[:3, :3] = r.as_matrix()
                T[:3, 3] = t
                poses.append(T)
    except FileNotFoundError:
        return False, [], []
    return True, color_image_files, poses


def px2cam(px):
    return np.array([(px[0] - CX) / FX, (px[1] - CY) / FY, 1])


def cam2px(p_cam):
    return np.array([p_cam[0] * FX / p_cam[2] + CX, p_cam[1] * FY / p_cam[2] + CY])


def inside(pt):
    return (pt[0] >= BOARDER + NCC_WINDOW_SIZE and pt[1] >= BOARDER + NCC_WINDOW_SIZE and
            pt[0] + BOARDER + NCC_WINDOW_SIZE < WIDTH and pt[1] + BOARDER + NCC_WINDOW_SIZE <= HEIGHT)


def get_bilinear_interpolated_value(img, pt):
    x = pt[0]
    y = pt[1]
    x1 = int(x)
    y1 = int(y)
    x2 = x1 + 1
    y2 = y1 + 1
    xx = x - x1
    yy = y - y1
    if x2 >= img.shape[1]:
        x2 = img.shape[1] - 1
    if y2 >= img.shape[0]:
        y2 = img.shape[0] - 1
    return ((1 - xx) * (1 - yy) * img[y1, x1] +
            xx * (1 - yy) * img[y1, x2] +
            (1 - xx) * yy * img[y2, x1] +
            xx * yy * img[y2, x2]) / 255.0


def NCC(ref, curr, pt_ref, pt_curr):
    mean_ref = 0
    mean_curr = 0
    values_ref = []
    values_curr = []
    for x in range(-NCC_WINDOW_SIZE, NCC_WINDOW_SIZE + 1):
        for y in range(-NCC_WINDOW_SIZE, NCC_WINDOW_SIZE + 1):
            value_ref = ref[int(pt_ref[1] + y), int(pt_ref[0] + x)] / 255.0
            mean_ref += value_ref
            value_curr = get_bilinear_interpolated_value(curr, pt_curr + np.array([x, y]))
            mean_curr += value_curr
            values_ref.append(value_ref)
            values_curr.append(value_curr)
    mean_ref /= NCC_AREA
    mean_curr /= NCC_AREA
    numerator = 0
    demoniator1 = 0
    demoniator2 = 0
    for i in range(len(values_ref)):
        n = (values_ref[i] - mean_ref) * (values_curr[i] - mean_curr)
        numerator += n
        demoniator1 += (values_ref[i] - mean_ref) ** 2
        demoniator2 += (values_curr[i] - mean_curr) ** 2
    return numerator / np.sqrt(demoniator1 * demoniator2 + 1e-10)


def epipolar_search(ref, curr, T_C_R, pt_ref, depth_mu, depth_cov):
    f_ref = px2cam(pt_ref)
    f_ref = f_ref / np.linalg.norm(f_ref)
    P_ref = f_ref * depth_mu
    px_mean_curr = cam2px(T_C_R[:3, :3] @ P_ref + T_C_R[:3, 3])
    d_min = max(depth_mu - 3 * depth_cov, 0.1)
    d_max = depth_mu + 3 * depth_cov
    px_min_curr = cam2px(T_C_R[:3, :3] @ (f_ref * d_min) + T_C_R[:3, 3])
    px_max_curr = cam2px(T_C_R[:3, :3] @ (f_ref * d_max) + T_C_R[:3, 3])
    epipolar_line = px_max_curr - px_min_curr
    epipolar_direction = epipolar_line / np.linalg.norm(epipolar_line)
    half_length = min(0.5 * np.linalg.norm(epipolar_line), 100)
    best_ncc = -1.0
    best_px_curr = None
    for l in np.arange(-half_length, half_length + 0.7, 0.7):
        px_curr = px_mean_curr + l * epipolar_direction
        if not inside(px_curr):
            continue
        ncc = NCC(ref, curr, pt_ref, px_curr)
        if ncc > best_ncc:
            best_ncc = ncc
            best_px_curr = px_curr
    if best_ncc < 0.85:
        return False, None
    return True, best_px_curr


def update_depth_filter(pt_ref, pt_curr, T_C_R, depth, depth_cov):
    T_R_C = np.linalg.inv(T_C_R)
    f_ref = px2cam(pt_ref)
    f_ref = f_ref / np.linalg.norm(f_ref)
    f_curr = px2cam(pt_curr)
    f_curr = f_curr / np.linalg.norm(f_curr)
    t = T_R_C[:3, 3]
    f2 = T_R_C[:3, :3] @ f_curr
    b = np.array([np.dot(f_ref, t), np.dot(f2, t)])
    A = np.array([[np.dot(f_ref, f_ref), -np.dot(f_ref, f2)],
                  [np.dot(f2, f_ref), -np.dot(f2, f2)]])
    d = np.linalg.det(A)
    lambdavec = np.linalg.inv(A) @ b
    xm = lambdavec[0] * f_ref
    xn = t + lambdavec[1] * f2
    d_esti = (xm + xn) / 2.0
    depth_estimation = np.linalg.norm(d_esti)
    p = f_ref * depth_estimation
    a = p - t
    t_norm = np.linalg.norm(t)
    a_norm = np.linalg.norm(a)
    alpha = np.arccos(np.dot(f_ref, t) / t_norm)
    beta = np.arccos(-np.dot(a, t) / (a_norm * t_norm))
    beta_prime = beta + np.arctan(1 / FX)
    gamma = np.pi - alpha - beta_prime
    p_prime = t_norm * np.sin(beta_prime) / np.sin(gamma)
    d_cov_est = p_prime - depth_estimation
    d_cov2 = d_cov_est ** 2
    mu = depth[int(pt_ref[1]), int(pt_ref[0])]
    sigma2 = depth_cov[int(pt_ref[1]), int(pt_ref[0])]
    mu_fuse = (d_cov2 * mu + sigma2 * depth_estimation) / (sigma2 + d_cov2)
    sigma_fuse2 = (sigma2 * d_cov2) / (sigma2 + d_cov2)
    depth[int(pt_ref[1]), int(pt_ref[0])] = mu_fuse
    depth_cov[int(pt_ref[1]), int(pt_ref[0])] = sigma_fuse2
    return True


def update(ref, curr, T_C_R, depth, depth_cov):
    for x in range(BOARDER, WIDTH - BOARDER):
        for y in range(BOARDER, HEIGHT - BOARDER):
            if depth_cov[y, x] < MIN_COV or depth_cov[y, x] > MAX_COV:
                continue
            ret, pt_curr = epipolar_search(ref, curr, T_C_R, np.array([x, y]), depth[y, x], np.sqrt(depth_cov[y, x]))
            if ret:
                update_depth_filter(np.array([x, y]), pt_curr, T_C_R, depth, depth_cov)
    return True


def plot_depth(depth):
    cv2.imshow("depth", depth * 0.4)
    cv2.waitKey(1)


if __name__ == "__main__":
    import sys
    if len(sys.argv) != 2:
        print("Usage: dense_mapping path_to_test_dataset")
        sys.exit(-1)
    path = sys.argv[1]
    ret, color_image_files, poses_TWC = read_dataset_files(path)
    if not ret:
        print("Reading image files failed!")
        sys.exit(-1)
    print(f"read total {len(color_image_files)} files.")
    ref = cv2.imread(color_image_files[0], 0)
    pose_ref_TWC = poses_TWC[0]
    init_depth = 3.0
    init_cov2 = 3.0
    depth = np.full((HEIGHT, WIDTH), init_depth, dtype=np.float64)
    depth_cov = np.full((HEIGHT, WIDTH), init_cov2, dtype=np.float64)
    for index in range(1, len(color_image_files)):
        print(f"*** loop {index} ***")
        curr = cv2.imread(color_image_files[index], 0)
        if curr is None:
            continue
        pose_curr_TWC = poses_TWC[index]
        pose_T_C_R = np.linalg.inv(pose_curr_TWC) @ pose_ref_TWC
        update(ref, curr, pose_T_C_R, depth, depth_cov)
        plot_depth(depth)
        cv2.imshow("image", curr)
        cv2.waitKey(1)
    print("estimation returns, saving depth map ...")
    cv2.imwrite("depth.png", depth)
    print("done.")

Reading image files failed!


SystemExit: -1

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
